In [5]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score as sil_score, f1_score, homogeneity_score
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(0)

def run_kmeans(X,y,title):

    kclusters = list(np.arange(2,50,2))
    sil_scores = []; f1_scores = []; homo_scores = []; train_times = []

    for k in kclusters:
        start_time = timeit.default_timer()
        km = KMeans(n_clusters=k, n_init=10,random_state=100,n_jobs=-1).fit(X)
        end_time = timeit.default_timer()
        train_times.append(end_time - start_time)
        sil_scores.append(sil_score(X, km.labels_))
        y_mode_vote = cluster_predictions(y,km.labels_)
        f1_scores.append(f1_score(y, y_mode_vote))
        homo_scores.append(homogeneity_score(y, km.labels_))
        
    # elbow curve for silhouette score
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(kclusters, sil_scores)
    plt.grid(True)
    plt.xlabel('No. Clusters')
    plt.ylabel('Avg Silhouette Score')
    plt.title('Elbow Plot for KMeans: '+ title)
    plt.show()
   
    # plot homogeneity scores
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(kclusters, homo_scores)
    plt.grid(True)
    plt.xlabel('No. Clusters')
    plt.ylabel('Homogeneity Score')
    plt.title('Homogeneity Scores KMeans: '+ title)
    plt.show()

    def evaluate_kmeans(km, X, y):
        start_time = timeit.default_timer()
        km.fit(X, y)
        end_time = timeit.default_timer()
        training_time = end_time - start_time
        
        y_mode_vote = cluster_predictions(y,km.labels_)
        auc = roc_auc_score(y, y_mode_vote)
        f1 = f1_score(y, y_mode_vote)
        accuracy = accuracy_score(y, y_mode_vote)
        precision = precision_score(y, y_mode_vote)
        recall = recall_score(y, y_mode_vote)
        cm = confusion_matrix(y, y_mode_vote)

        print("Model Evaluation Metrics Using Mode Cluster Vote")
        print("*****************************************************")
        print("Model Training Time (s):   "+"{:.2f}".format(training_time))
        print("No. Iterations to Converge: {}".format(km.n_iter_))
        print("F1 Score:  "+"{:.2f}".format(f1))
        print("Accuracy:  "+"{:.2f}".format(accuracy)+"     AUC:       "+"{:.2f}".format(auc))
        print("Precision: "+"{:.2f}".format(precision)+"     Recall:    "+"{:.2f}".format(recall))
        print("*****************************************************")
        plt.figure()
        plot_confusion_matrix(cm, classes=["0","1"], title='Confusion Matrix')
        plt.show()